In [1]:
import pandas as pd

In [2]:
dataframe = pd.read_csv('/home/deepak/Documents/AMZN.csv')

In [3]:
data = dataframe.copy()

In [4]:
data.shape
data.columns

Index(['Date', 'Close Price', 'Open Price', 'High Price', 'Low Price',
       'Total Traded Quantity'],
      dtype='object')

In [5]:
data.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
0,"Oct 24, 2019","1,780.78","1,770.78","1,787.97","1,760.78",5.20M
1,"Oct 23, 2019","1,762.17","1,761.30","1,770.05","1,742.00",2.19M
2,"Oct 22, 2019","1,765.73","1,788.15","1,789.78","1,762.00",2.23M
3,"Oct 21, 2019","1,785.66","1,769.66","1,785.88","1,765.00",2.22M
4,"Oct 18, 2019","1,757.51","1,787.80","1,793.98","1,749.20",3.37M


In [20]:
data.dtypes
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   503 non-null    datetime64[ns]
 1   Close Price            503 non-null    float64       
 2   Open Price             503 non-null    float64       
 3   High Price             503 non-null    float64       
 4   Low Price              503 non-null    float64       
 5   Total Traded Quantity  503 non-null    float64       
 6   Month                  503 non-null    int64         
 7   Year                   503 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 31.6 KB


In [7]:
def Stringtofloat(val):
    new = val.replace(',','')
    return float(new)
def TTQ(val):
    new = val.replace('M','')
    return float(new)

In [19]:
# data['Close Price'] = data['Close Price'].apply(Stringtofloat)
# data['Open Price'] = data['Open Price'].apply(Stringtofloat)
# data['High Price'] = data['High Price'].apply(Stringtofloat)
# data['Open Price'] = data['Open Price'].apply(Stringtofloat)
# data['Low Price'] = data['Low Price'].apply(Stringtofloat)
# data['Total Traded Quantity'] = data['Total Traded Quantity'].apply(TTQ)
# data['Date'] = data['Date'].astype('datetime64[ns]')

In [9]:
data['Close Price'][:90].describe()

count           90
unique          90
top       1,939.00
freq             1
Name: Close Price, dtype: object

In [10]:
data['Date']

0      Oct 24, 2019
1      Oct 23, 2019
2      Oct 22, 2019
3      Oct 21, 2019
4      Oct 18, 2019
           ...     
498    Oct 31, 2017
499    Oct 30, 2017
500    Oct 27, 2017
501    Oct 26, 2017
502    Oct 25, 2017
Name: Date, Length: 503, dtype: object

In [21]:
#create dataframe column month and year using existing Date Column
data['Month'] = pd.DatetimeIndex(data['Date']).month
data['Year'] = pd.DatetimeIndex(data['Date']).year
data.head()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year
0,2019-10-24,1780.78,1770.78,1787.97,1760.78,5.20,10,2019
1,2019-10-23,1762.17,1761.30,1770.05,1742.00,2.19,10,2019
2,2019-10-22,1765.73,1788.15,1789.78,1762.00,2.23,10,2019
3,2019-10-21,1785.66,1769.66,1785.88,1765.00,2.22,10,2019
4,2019-10-18,1757.51,1787.80,1793.98,1749.20,3.37,10,2019


In [22]:
# vwap = sum(closing_price*vol)/sum(vol)
# VWAP = sum(data['Close Price']*data['Total Traded Quantity'])/sum(data['Total Traded Quantity'])

forvwap = data.groupby(['Month']).sum()

VWAP = [forvwap['Close Price'][i]*forvwap['Total Traded Quantity'][i]/forvwap['Total Traded Quantity'][i] for i in range(1,13)]
print(VWAP)

[61929.909999999996, 58316.70999999999, 68520.05, 70022.88000000002, 76214.31000000001, 72727.65, 80727.04000000002, 83110.77999999998, 73337.90000000001, 77695.61999999998, 58071.12999999998, 53006.25]


In [23]:
def average_price(days):
    return data['Close Price'][:days].mean()
def profit_loss(days):
    p = 0
    for i in range(days):
        p += (data['Close Price'][i+1] - data['Close Price'][i])/100
    return p

In [24]:
data['Day_perc_Change'] = data['Close Price'].pct_change()

In [25]:
data['Day_perc_Change'].dropna()

1     -0.010450
2      0.002020
3      0.011287
4     -0.015764
5      0.017053
         ...   
498    0.001450
499    0.005039
500   -0.008912
501   -0.116736
502    0.000494
Name: Day_perc_Change, Length: 502, dtype: float64

In [27]:
data['Day_perc_Change'] = data['Day_perc_Change']*100

In [54]:
i = 0
for change in data['Day_perc_Change']:
    if -0.5 < change < 0.5:
        data['Trend'][i] = 'Slight or No change'
    if 0.5 < change < 1:
        data['Trend'][i] = 'Slight positive'
    if -1 < change < -0.5:
        data['Trend'][i] = 'Slight negative'
    if 1< change <3:
        data['Trend'][i] = 'Positive'
    if -3 < change <-1:
        data['Trend'][i] = 'Negative'
    if 3 < change < 7:
        data['Trend'][i] = 'Among top gainers'
    if -7< change <-3:
        data['Trend'][i] = 'Among top losers'
    if change > 7:
        data['Trend'][i] = 'Bull run'
    if change < -7:
        data['Trend'][i] = 'Bear drop'
    i += 1 

<ipython-input-54-7275d446a43e>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'][i] = 'Negative'
<ipython-input-54-7275d446a43e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'][i] = 'Slight or No change'
<ipython-input-54-7275d446a43e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Trend'][i] = 'Positive'
<ipython-input-54-7275d446a43e>:8: SettingWithCopyWarning: 
A value is trying to be set on a 

In [ ]:
data.columns

In [55]:
data['Trend'].head(20)

0     Slight or No change
1                Negative
2     Slight or No change
3                Positive
4                Negative
5                Positive
6         Slight negative
7         Slight negative
8                Negative
9     Slight or No change
10        Slight negative
11    Slight or No change
12        Slight negative
13               Positive
14    Slight or No change
15        Slight negative
16        Slight negative
17               Positive
18    Slight or No change
19        Slight negative
Name: Trend, dtype: object

In [56]:
cal = data.groupby(data['Trend'])
avg = cal['Total Traded Quantity'].mean()
median = cal['Total Traded Quantity'].median()
print(avg, median)


Trend
Among top gainers       6.445161
Among top losers        7.190000
Bear drop               6.420000
Bull run               10.290000
Negative                4.891727
Positive                5.060746
Slight negative         4.368082
Slight or No change     4.188966
Slight positive         4.620192
Name: Total Traded Quantity, dtype: float64 Trend
Among top gainers       6.020
Among top losers        6.515
Bear drop               6.420
Bull run               10.290
Negative                4.385
Positive                4.520
Slight negative         3.810
Slight or No change     3.680
Slight positive         4.265
Name: Total Traded Quantity, dtype: float64


In [57]:
data.to_csv('/home/deepak/Documents/week2.copy.csv',index = False)

In [ ]:
a = [1,2,3]
b =[2,3,5]
x = [for i in b if i not in a]